# Кластеризация: Размещение баннеров

In [1]:
import pandas as pd
import numpy as np

##### Подгрузим данные о скоплении туристов по географическим координатам

In [3]:
data = pd.read_csv("checkins.csv", header=0, sep='|')

In [4]:
data.head(5)

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,,,2012-04-21 17:42:58


In [5]:
# выведим название столбцов
data.columns

Index([u'   id    ', u' user_id ', u' venue_id ', u'     latitude      ',
       u'     longitude     ', u'     created_at      '],
      dtype='object')

In [6]:
# удаление лишних стобцов
data.drop(['   id    ',' user_id ',' venue_id ','     created_at      '], axis=1, inplace=True)

In [7]:
data.head(5)

,latitude,longitude
0,,
1,38.8951118,-77.0363658
2,,
3,33.800745,-84.41052
4,,


In [8]:
# выкидывем строки с пустыми ячейками
data = data.replace(r'^\s+$', np.nan, regex=True).dropna() 

In [9]:
data.head(5)

,latitude,longitude
1,38.8951118,-77.0363658
3,33.800745,-84.41052
7,45.5234515,-122.6762071
9,40.764462,-111.904565
10,33.4483771,-112.0740373


In [10]:
# перемешаем строки в таблице
from sklearn.utils import shuffle
data = shuffle(data)

In [11]:
data.head(5)

,latitude,longitude
448621,42.48059,-83.4754913
11654,31.8456816,-102.3676431
512840,30.3321838,-81.655651
65501,33.4941704,-111.9260519
310254,36.1988592,-115.1175013


In [12]:
data.shape

(396634, 2)

In [13]:
# выберем для анализа первые 100 000 строк
data = data[:100000]
data.shape

(100000, 2)

##### Строим модель кластеризации MeanShift

In [14]:
from sklearn.cluster import MeanShift

ms = MeanShift(bandwidth=0.1) # создаем модель
ms.fit(data) # обучаем модель на нашей выборке

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [15]:
# получение сгенерированных целевых меток (y) для всех элементов выборки (мы сами создаем целевые метки для каждого элемента)
labels = ms.labels_
# получение центров кластеров (в порядке меток labels)
cluster_centers = ms.cluster_centers_
print labels.shape
print cluster_centers.shape

(100000,)
(3636, 2)


In [64]:
#подсчет объема кластеров и отбор больших (в которых элементов больше 15)

# создаем счетчик по целевым меткам (подсчитаем количетсов элементов для каждого кластера - целевой метки labels)
import collections
counter = collections.Counter(labels)

# filter возвращает элементы из counter.items() в которых элемент[1]>15
result = filter(lambda x: x[1]>15, counter.items()) # result - массив состоящий из (класс, количетсво)
labels_bigclusters = zip(*result) # zip(*result) распаковывает массив result (поворачивает матрицу на 90 градусов)
print len(labels_bigclusters[0]) # всего у нас 609 кластеров

609


In [65]:
#загрузка Офисов, в разных городах, из задания по координатам latitude (широта) и longitude (долгота)
carivalOffice = pd.DataFrame({'latitude':[33.751277, 25.867736, 51.503016, 52.378894, 39.366487, -33.868457],
                              'longitude':[-118.188740, -80.324116, -0.075479, 4.885084, 117.036146, 151.205134]})

In [66]:
carivalOffice.shape

(6, 2)

In [81]:
#ищем 20 кластерных центров наиболее близких к офисам
result_dist = []
countertmp = 0

from numpy import linalg

for tmp in cluster_centers: # цикл по центрам кластеров (координатам)
    if countertmp in labels_bigclusters[0]: # если такой кластер существует (мы его не удалили)
        # тогда добавь в массив: данная целева метка кластера, дистанция от центра кластера до Ближайшего Офиса (min) 
        result_dist.append ([countertmp, min(linalg.norm(carivalOffice - tmp, axis=1))])  
    countertmp += 1
    
print result_dist[:20]
# Упорядочим элементы словаря по первому элементу (дистанции)
result_dist = sorted(result_dist, key=lambda distance: distance[1])
print result_dist[:20]

[[0, 16.144646053158699], [1, 17.598479662824378], [2, 6.2938142711283929], [3, 6.1221520997406769], [4, 13.424171651463816], [5, 5.7707575539552298], [6, 5.8494294363827066], [7, 8.8868153711422586], [8, 18.911710128922916], [9, 14.461240793690903], [10, 0.32283987744148179], [11, 6.3785289092096393], [12, 3.8344733650684897], [13, 23.080729367116763], [14, 17.965936946793459], [15, 15.000052325324734], [16, 16.252502182158793], [17, 1.3513434339175854], [18, 0.3770118395450533], [19, 14.494116289068934]]
[[243, 0.0013999018759021488], [279, 0.0097628089753440134], [52, 0.052101984401796024], [208, 0.066447991387102875], [50, 0.072808064284032678], [23, 0.13604952649957955], [94, 0.18360902195019474], [83, 0.19548226212119962], [49, 0.26844818184191882], [33, 0.2879918285307409], [54, 0.28873222567481893], [32, 0.30129623076146728], [10, 0.32283987744148179], [34, 0.35402563731314485], [79, 0.36681094440145873], [18, 0.3770118395450533], [56, 0.38265943582281103], [118, 0.386063572025

In [82]:
for tmp in result_dist[:20]:
    print "index %s - center %s - dist %s - power %s" % (tmp[0],cluster_centers[tmp[0]], tmp[1],labels_bigclusters[1][tmp[0]] )

index 243 - center [ -33.86706468  151.20498854] - dist 0.0013999018759 - power 54
index 279 - center [ 52.37177154   4.89176105] - dist 0.00976280897534 - power 48
index 52 - center [ 51.5028701   -0.12758078] - dist 0.0521019844018 - power 319
index 208 - center [ 25.91402968 -80.27644808] - dist 0.0664479913871 - power 76
index 50 - center [  33.81085567 -118.14689031] - dist 0.072808064284 - power 335
index 23 - center [ 25.78676198 -80.21478768] - dist 0.1360495265 - power 700
index 94 - center [ 26.00653795 -80.20392327] - dist 0.18360902195 - power 122
index 83 - center [  33.90025783 -118.06217766] - dist 0.195482262121 - power 160
index 49 - center [  33.64203986 -117.94352239] - dist 0.268448181842 - power 326
index 33 - center [  33.93872277 -118.40737983] - dist 0.287991828531 - power 424
index 54 - center [ 26.10467683 -80.15911496] - dist 0.288732225675 - power 227
index 32 - center [  33.8102705  -117.89327564] - dist 0.301296230761 - power 467
index 10 - center [  34.06

In [83]:
# запишем ответ в файл
with open("answer.txt", "w") as F:
    F.write("-33.86792497  151.20815811")